In [ ]:
%env
SCHEMA=https://
OUTPUT=s3://
ETL_CONF_ENV=production

In [ ]:
{
  "type": "DelimitedExtract",
  "name": "extract csv data from nyc_tripdata",
  "environments": ["production", "test"],
  "inputURI": "s3a://nyc-tlc/csv_backup/green_tripdata_*.csv",
  "outputView": "green_tripdata0_raw",            
  "delimiter": "Comma",
  "quote" : "DoubleQuote",
  "header": true,
  "persist": true
}

In [ ]:
{
  "type": "TypingTransform",
  "name": "apply green_tripdata schema 0 data types",
  "environments": ["production", "test"],
  "schemaURI": ${SCHEMA},
  "inputView": "green_tripdata0_raw",            
  "outputView": "green_tripdata0"
}

In [ ]:
%sql name="aggregate the result by month and year" outputView=green_trip_summery environments=production,test persist=true

SELECT 
   year(lpep_pickup_datetime) AS trip_year
  ,month(lpep_pickup_datetime) AS trip_month
  ,vendor_id
  ,sum(coalesce(trip_distance,0)) AS total_distance
  ,sum(coalesce(total_amount,0)) AS total_fee
FROM green_tripdata0
GROUP BY trip_year, trip_month, vendor_id

In [ ]:
{
  "type": "ParquetLoad",
  "name": "write out green_tripdata0 dataset as Parquet",
  "environments": ["production", "test"],
  "inputView": "green_trip_summery",
  "outputURI": ${OUTPUT},
  "saveMode": "Overwrite"
}